# Project specification


## 1) Calibration (camera matrix, distortion coefficients)

## Pipeline
* 2) Color transform, Gradients -> thresholded binary image
* 3) Perspective transform
* 4) Identification of lane line pixels
* 5) Radius of curvature calculation
* 6) Lane area plotted back down onto the road

* (7) Lane history + smoothing)

# Imported modules

# 1) Calibration


Theory

### todo: _intrinsic or extrinsic?!_ -> distortion correction

"For better results, we need atleast 10 test patterns."
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html

"Now for X,Y values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of points. In this case, the results we get will be in the scale of size of chess board square." https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html

...

---

* a) Prepare object points for chessboard corners
* b) Find chessboard corners in all images
* c) Calibrate camera (distortion correction) https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_calibration/py_calibration.html
* d) get perspective transform

In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from help_func import plot, draw_lines_from_points
import pickle
# for interactive plots
#%matplotlib notebook
%matplotlib inline


### moved to calibration.ipynb

In [10]:
# # initialize array for object points with dimensions (6*9, 3)
# objp = np.zeros((6*9,3), np.float32)
# # create mesh grid, transpose and reshape to get an (6*9, 2) array 
# # for all object points (z component is assumed to be 0)
# objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# # initialize arrays for object- and image points from all
# # calibration images
# objpts = [] # 3D points in real world space
# imgpts = [] # 2D points in image plane

### moved to calibration.ipynb

In [11]:
# # get list of all available calibration images
# cal_images = glob.glob('camera_cal/calibration*.jpg')

### moved to calibration.ipynb

In [12]:
# # find chessboard corners in all images
# for idx, img_name in enumerate(cal_images):
#     img = cv2.imread(img_name) # read image
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    
#     # find chessboard corners
#     ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
#     # if corners were found, object- and image points are added
#     if ret == True:
#         objpts.append(objp)
#         imgpts.append(corners)

#         # draw and display/save corners
# #         cv2.drawChessboardCorners(img, (9,6), corners, ret)        
# #         folder_name = 'output_images/'
# #        # file_name = 'chessboard_corners_'+img_name[11:]
# #        # cv2.imwrite(folder_name+file_name, img)
#         # cv2.imshow('img', img)
#         # cv2.waitKey(500)

idea for calibration function:
https://github.com/mithi/advanced-lane-detection

In [13]:
### todo: class and functions descriptions
class Calibration:
    
    def __init__(self, cam_mtx, dist_coeff, src_pts, dst_pts):
        self.cam_mtx = cam_mtx
        self.dist_coeff = dist_coeff
        self.src_pts = src_pts
        self.dst_pts = dst_pts
        self.warp_mtx = cv2.get_PerspectiveTransform(scr_pts, dst_pts)
        
    def undistort(self, dist_img):
        undist_img = cv2.undistort(dist_img, self.cam_mtx, 
                                   self.dist_coeff, None, 
                                   self.cam_mtx)
        return undist_img
    
    def warp(self, img):
        img_size = (img.shape[1], img.shape[0])
        warped_img = cv2.warpPerspective(img, self.warp_mtx, 
                                         img_size, 
                                         flags = cv2.INTER_LINEAR)
        return warped_img

### moved to calibration.ipynb

In [14]:
# # load distorted image and get size
# dist_img = cv2.imread('camera_cal/calibration2.jpg')
# img_size = (img.shape[1], img.shape[0])

# # calibrate camera (returns the camera matrix, distortion coefficients, rotation and translation vectors)
# ret, mtx, dist_coeff, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, img_size, None ,None)

# calib = Calibration(mtx, dist_coeff, src_pts, dst_pts)

# undist_img = calib.undistort(dist_img)

# # undistort example image
# # undist = cv2.undistort(dist_img, mtx, dist_coeff, None, mtx)
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()

# ax1.imshow(dist_img)
# ax2.imshow(undist_img)

# # plt.imshow(undist_img)
# # cv2.imwrite('output_images/calibration2_undistorted.jpg', undist)

In [15]:
pickled_data = pickle.load(open("pickled_calib_data.p", "rb"))
mtx = pickled_data["mtx"]
dist_coeff = pickled_data["dist_coeff"]
warp_mtx = pickled_data["warp_mtx"]

---

# Pipeline

## Create binary image

### gradient (function sobel_thresh)

>"In our last example, output datatype is cv2.CV_8U or np.uint8. But there is a slight problem with that. Black-to-White transition is taken as Positive slope (it has a positive value) while White-to-Black transition is taken as a Negative slope (It has negative value). So when you convert data to np.uint8, all negative slopes are made zero. In simple words, you miss that edge.

>If you want to detect both edges, better option is to keep the output datatype to some higher forms, like cv2.CV_16S, cv2.CV_64F etc, take its absolute value and then convert back to cv2.CV_8U. Below code demonstrates this procedure for a horizontal Sobel filter and difference in results."

https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_gradients/py_gradients.html

In [ ]:
def grad_thresh(img, orient='x', kernel=3, sobel_thresh=(0, 255)):
    """Creates thresholded binary image based on directional 
    gradient"""
    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # take derivative in 'orient' direction
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel)
    else:
        print('only x and y are accepted as 2nd argument')
        return None
    # calculate absolute value and scale to 8-bit
    abs_sobel = np.abs(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel/np.max(abs_sobel))
    # create binary image
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & 
                scaled_sobel <= thresh[1]] = 1
    # return binary image
    return grad_binary

---

In [ ]:
# src_img = mpimg.imread('test_images/straight_lines1.jpg')

# img_size = (src_img.shape[1], src_img.shape[0])

# undist_img = calib.undistort(src_img)

In [ ]:
# src_pts determined manually based on the test image straight_lines1.jpg
# src_pts = np.array([[560, 475], 
#                     [725, 475], 
#                     [1100, img_size[1]], 
#                     [200, img_size[1]]], 
#                    np.float32).reshape((4, 1, 2))
# currently chosen:
# src_pts = np.array([[580, 460], 
#                     [700, 460], 
#                     [1100, img_size[1]], 
#                     [200, img_size[1]]], 
#                    np.float32).reshape((4, 1, 2))
# src_pts = np.array([[595, 450], 
#                     [685, 450], 
#                     [1100, img_size[1]], 
#                     [200, img_size[1]]], 
#                    np.float32).reshape((4, 1, 2))

In [ ]:
# # destination points for perspective transformation
# lane_dist = 700 # in pixels
# mid_pnt = img_size[0]//2
# print(mid_pnt)
# # dst_pts = [(mid_pnt - lane_dist, 0), (mid_pnt + lane_dist, 0), 
# #           (mid_pnt + lane_dist, img_size[1]), 
# #            (mid_pnt - lane_dist, img_size[0])]
# dst_pts = np.array([[mid_pnt - lane_dist/2, 0], 
#                     [mid_pnt + lane_dist/2, 0], 
#                     [mid_pnt + lane_dist/2, img_size[1]],
#                     [mid_pnt - lane_dist/2, img_size[1]]], 
#                    np.float32).reshape((4, 1, 2))

In [ ]:
# line1 = np.array([src_pts[0,:], src_pts[1,:]]).reshape(1, 4)
# line2 = np.array([src_pts[1,:], src_pts[2,:]]).reshape(1, 4)
# line3 = np.array([src_pts[2,:], src_pts[3,:]]).reshape(1, 4)
# line4 = np.array([src_pts[3,:], src_pts[0,:]]).reshape(1, 4)
# lines = np.vstack([line1, line2, line3, line4]).reshape((4,1,4))
# # print(lines.shape)
# undist_img_cp = np.copy(undist_img)
# draw_lines(undist_img_cp, lines)
# plot(undist_img_cp)

In [ ]:
# unwarped_img = np.copy(undist_img)
raw_img = mpimg.imread('test_images/test6.jpg')
undist_img = calib.undistort(raw_img)
# raw_img = mpimg.imread('test_images/test1.jpg')
# undist_img = calib.undistort(raw_img)

warp_mtx = cv2.getPerspectiveTransform(src_pts, dst_pts)
warped_img = cv2.warpPerspective(undist_img, warp_mtx, 
                                 img_size, flags=cv2.INTER_LINEAR)

plt.figure()
plt.imshow(undist_img)
line_img1 = draw_lines_from_points(undist_img, src_pts)
plt.figure()
plt.imshow(undist_img)

plt.figure()
plt.imshow(warped_img)
line_img2 = draw_lines_from_points(warped_img, dst_pts)
plt.figure()
plt.imshow(warped_img)


In [ ]:
src_pts

In [ ]:
dst_pts

In [ ]:
def pipeline(src_img):
    # undistort image    
    
    undist_img = calib.undistort(src_img)
#     plot_func(img); plot_func(undist)

#     warped_img = calib.warp(undist)
    warp_mtx = cv2.getPerspectiveTransform(src_pts, )

    plot(undist_img)
    plot(warped_img)
    
    #     grad_thresh(undist)

In [ ]:
img = mpimg.imread('test_images/straight_lines1.jpg')

pipeline(img)